## Imports/Setup

In [2]:
import pandas as pd
import os
import re
import numpy as np
from datetime import datetime, timedelta

def parse_custom_time(time_str):
    """
    Parses a time string like '450P' or '125A' and returns a datetime.time object.
    """
    if pd.isna(time_str) or time_str == '*':
        return pd.NaT
    
    time_str = str(time_str)
    am_pm = time_str[-1].upper()
    time_val = time_str[:-1]

    # Handle cases where the time is a single digit hour
    if len(time_val) == 3:
        hour = int(time_val[0])
        minute = int(time_val[1:])
    else:
        hour = int(time_val[:2])
        minute = int(time_val[2:])

    # Adjust hour for AM/PM logic
    if am_pm == 'P' and hour != 12:
        hour += 12
    elif am_pm == 'A' and hour == 12:  # Special case for 12 AM
        hour = 0
    
    return pd.to_datetime(f"{hour:02d}:{minute:02d}", format='%H:%M', errors='coerce').time()

## Read in exceptions list from previous steps

In [3]:
all_exceptions = pd.read_csv("./exceptions_full.csv")

## Look at a file of each type to understand the error

In [4]:
all_exceptions['error'].value_counts()

error
invalid literal for int() with base 10: 'SE'    1148
invalid literal for int() with base 10: 'CA'     808
invalid literal for int() with base 10: 'b'      328
invalid literal for int() with base 10: 'rt'       2
Name: count, dtype: int64

In [5]:
for err in all_exceptions['error'].unique():
    error_files = list(all_exceptions['filepath'].loc[all_exceptions['error'] == err])
    first_file = error_files[0]
    print(err, first_file)
    # with open(first_file, 'r') as f:
    #     for line in f:
    #         print(line)

invalid literal for int() with base 10: 'SE' ../data/2-csv/2011/66/66_20110722.csv
invalid literal for int() with base 10: 'CA' ../data/2-csv/2014/66/66_20140728.csv
invalid literal for int() with base 10: 'rt' ../data/2-csv/2015/132/132_20150503.csv
invalid literal for int() with base 10: 'b' ../data/2-csv/2021/66/66_20210923.csv


## Error File Inspection Findings
* `invalid literal for int() with base 10: 'SE'`
    * this is a service disruption - first row in CSV after header says: THIS,TRAIN,EXPERIENCED,A,SERVICE,DISRUPTION.,,
* `invalid literal for int() with base 10: 'CA'`
    * this is a cancellation - first row in CSV after header says: THIS,TRAIN,HAS,EXPERIENCED,CANCELLATIONS.,,,
* `invalid literal for int() with base 10: 'b'`
    * this file was created by chris juckins - first row in CSV after header says: NOTE:,This,file,created,by,juckins.net/asmad,,
* `invalid literal for int() with base 10: 'rt'`
    * idk what's wrong with these files lol let's ignore them because it's only 2

## Handle the three main error types separately

### Process Error Type 3: `invalid literal for int() with base 10: 'b'`
This one is easy as we just need to remove the first row. 

In [6]:
exceptions = [] 

In [9]:
err = "invalid literal for int() with base 10: 'b'"
error_files = list(all_exceptions['filepath'].loc[all_exceptions['error'] == err])

for filepath in error_files:
    try:
        filename = filepath.split("/")[-1]
        train_number_extracted = filename.split("_")[0]
        date_string = filename.split("_")[1].rstrip(".csv")
        date_parsed = datetime.strptime(date_string, "%Y%m%d").date()
        output_dir = f"data/3-augmented/{date_parsed.year}/{train_number_extracted}"
        df = pd.read_csv(filepath)
        df = df.drop(0, axis=0)

        # Augment with columns containing actual date of departure and arrival 
        origination_date = pd.to_datetime(date_parsed)

        df['Origin Date'] = origination_date
        df['Train Number'] = train_number_extracted
        df['Service Disruption'] = 0
        df['Cancellation'] = 0
        
        departure_day_offset = pd.to_numeric(df['Schedule Departure Day'], errors='coerce') - 1
        arrival_day_offset = pd.to_numeric(df['Schedule Arrival Day'], errors='coerce') - 1

        df['Scheduled Departure Date'] = origination_date + pd.to_timedelta(departure_day_offset, unit='d')
        df['Scheduled Arrival Date'] = origination_date + pd.to_timedelta(arrival_day_offset, unit='d')

        # Augment with columns for actual datetime 
        df['Parsed Schedule Departure Time'] = df['Schedule Departure Time'].apply(parse_custom_time)
        df['Parsed Schedule Arrival Time'] = df['Schedule Arrival Time'].apply(parse_custom_time)
        df['Parsed Actual Departure Time'] = df['Actual Departure Time'].apply(parse_custom_time)
        df['Parsed Actual Arrival Time'] = df['Actual Arrival Time'].apply(parse_custom_time)
        
        df['Scheduled Departure Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Departure Date']).split()[0] + ' ' + str(row['Parsed Schedule Departure Time'])) 
            if pd.notna(row['Scheduled Departure Date']) and pd.notna(row['Parsed Schedule Departure Time'])
            else pd.NaT, axis=1)
        
        df['Scheduled Arrival Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Arrival Date']).split()[0] + ' ' + str(row['Parsed Schedule Arrival Time'])) 
            if pd.notna(row['Scheduled Arrival Date']) and pd.notna(row['Parsed Schedule Arrival Time'])
            else pd.NaT, axis=1)
        
        df['Actual Departure Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Departure Date']).split()[0] + ' ' + str(row['Parsed Actual Departure Time'])) 
            if pd.notna(row['Scheduled Departure Date']) and pd.notna(row['Parsed Actual Departure Time'])
            else pd.NaT, axis=1)
        
        df['Actual Arrival Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Arrival Date']).split()[0] + ' ' + str(row['Parsed Actual Arrival Time'])) 
            if pd.notna(row['Scheduled Arrival Date']) and pd.notna(row['Parsed Actual Arrival Time'])
            else pd.NaT, axis=1)

        # Output cleaned + augmented data
        df = df[['Origin Date', 'Train Number', 'Service Disruption', 'Cancellation', 'Station Code','Scheduled Departure Datetime', 'Scheduled Arrival Datetime', 'Actual Departure Datetime', 'Actual Arrival Datetime', 'Comments']]
        output_filepath = os.path.join(output_dir, filename)                   
        df.to_csv(output_filepath, index=False)
    except ValueError as e:
        exceptions.append([filepath, e])
    except IndexError as e:
        exceptions.append([filepath, e])
    

In [10]:
len(exceptions) # this is fine, moving on 

1

### Process Error Type 2: `invalid literal for int() with base 10: 'CA'` aka Stop Cancellation(s) Occurred

In [11]:
def stop_was_cancelled(val):
    if val == 'Stop':
        return 1
    else:
        return 0

In [12]:
exceptions_2 = []

err = "invalid literal for int() with base 10: 'CA'"
error_files = list(all_exceptions['filepath'].loc[all_exceptions['error'] == err])

for filepath in error_files:
    try:
        filename = filepath.split("/")[-1]
        train_number_extracted = filename.split("_")[0]
        date_string = filename.split("_")[1].rstrip(".csv")
        date_parsed = datetime.strptime(date_string, "%Y%m%d").date()
        output_dir = f"data/3-augmented/{date_parsed.year}/{train_number_extracted}"
        
        df = pd.read_csv(filepath)
        df = df.drop(df.loc[df['Station Code'] == 'THIS'].index, axis=0)

        # Add SD + C fields 
        df['Cancellation'] = df['Actual Departure Time'].apply(stop_was_cancelled)
        df['Service Disruption'] = ""

        # Lazy cleaning
        df = df.replace('Stop', pd.NaT)
        df = df.replace('Station', pd.NaT)
        df = df.replace('ET', pd.NaT)
        df = df.replace('SuMoTuWeTh',pd.NaT )
        df = df.replace('MoTuWeTh', pd.NaT)
        df = df.replace('MoTuWeThFr', pd.NaT)
        df = df.replace('SuMoTuWeThFr', pd.NaT)
        df = df.replace('Sa', pd.NaT)
        df = df.replace('SaSu', pd.NaT)
        df = df.replace('Su', pd.NaT)
    
        # Augment with columns containing actual date of departure and arrival 
        origination_date = pd.to_datetime(date_parsed)
        
        df['Origin Date'] = origination_date
        df['Train Number'] = train_number_extracted
        
        departure_day_offset = pd.to_numeric(df['Schedule Departure Day'], errors='coerce') - 1
        arrival_day_offset = pd.to_numeric(df['Schedule Arrival Day'], errors='coerce') - 1
    
        df['Scheduled Departure Date'] = origination_date + pd.to_timedelta(departure_day_offset, unit='d')
        df['Scheduled Arrival Date'] = origination_date + pd.to_timedelta(arrival_day_offset, unit='d')
    
        # Augment with columns for actual datetime 
        df['Parsed Schedule Departure Time'] = df['Schedule Departure Time'].apply(parse_custom_time)
        df['Parsed Schedule Arrival Time'] = df['Schedule Arrival Time'].apply(parse_custom_time)
        df['Parsed Actual Departure Time'] = df['Actual Departure Time'].apply(parse_custom_time)
        df['Parsed Actual Arrival Time'] = df['Actual Arrival Time'].apply(parse_custom_time)
        
        df['Scheduled Departure Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Departure Date']).split()[0] + ' ' + str(row['Parsed Schedule Departure Time'])) 
            if pd.notna(row['Scheduled Departure Date']) and pd.notna(row['Parsed Schedule Departure Time'])
            else pd.NaT, axis=1)
        
        df['Scheduled Arrival Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Arrival Date']).split()[0] + ' ' + str(row['Parsed Schedule Arrival Time'])) 
            if pd.notna(row['Scheduled Arrival Date']) and pd.notna(row['Parsed Schedule Arrival Time'])
            else pd.NaT, axis=1)
        
        df['Actual Departure Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Departure Date']).split()[0] + ' ' + str(row['Parsed Actual Departure Time'])) 
            if pd.notna(row['Scheduled Departure Date']) and pd.notna(row['Parsed Actual Departure Time'])
            else pd.NaT, axis=1)
        
        df['Actual Arrival Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Arrival Date']).split()[0] + ' ' + str(row['Parsed Actual Arrival Time'])) 
            if pd.notna(row['Scheduled Arrival Date']) and pd.notna(row['Parsed Actual Arrival Time'])
            else pd.NaT, axis=1)
    
        # Output cleaned + augmented data
        df = df[['Origin Date', 'Train Number', 'Service Disruption', 'Cancellation', 'Station Code','Scheduled Departure Datetime', 'Scheduled Arrival Datetime', 'Actual Departure Datetime', 'Actual Arrival Datetime', 'Comments']]
        output_filepath = os.path.join(output_dir, filename)                   
        df.to_csv(output_filepath, index=False)
    except ValueError as e:
        exceptions_2.append([filepath, e])
        print(df)
    except IndexError as e:
        exceptions_2.append([filepath, e])
    

   Station Code Schedule Arrival Day Schedule Arrival Time  \
1           BOS                    *                     *   
2           BBY                    *                     *   
3           RTE                    *                     *   
4           PVD                    *                     *   
5           KIN                    *                     *   
6           WLY                    *                     *   
7           MYS                    *                     *   
8           NLC                    *                     *   
9           OSB                    *                     *   
10          NHV                    1                 1149A   
11          BRP                    *                     *   
12          STM                    *                     *   
13          NRO                    *                     *   
14          NYP                    1                  148P   
15          NWK                    *                     *   
16      

In [13]:
len(exceptions_2)

1

In [14]:
exceptions_2 # ok whatever let's move on

[['../data/2-csv/2023/93/93_20230828.csv',
  ValueError("invalid literal for int() with base 10: ''")]]

### Process Error Type 1: `invalid literal for int() with base 10: 'SE'` aka Service Disruption occurred

In [15]:
exceptions_1 = []

err = "invalid literal for int() with base 10: 'SE'"
error_files = list(all_exceptions['filepath'].loc[all_exceptions['error'] == err])

for filepath in error_files:
    try:
        filename = filepath.split("/")[-1]
        train_number_extracted = filename.split("_")[0]
        date_string = filename.split("_")[1].rstrip(".csv")
        date_parsed = datetime.strptime(date_string, "%Y%m%d").date()
        output_dir = f"data/3-augmented/{date_parsed.year}/{train_number_extracted}"
        
        df = pd.read_csv(filepath)
        df = df.drop(df.loc[df['Station Code'] == 'THIS'].index, axis=0)

        # Add SD + C fields 
        df['Cancellation'] = df['Actual Departure Time'].apply(stop_was_cancelled)
        df['Service Disruption'] = 1

        # Lazy cleaning
        df = df.replace('Stop', pd.NaT)
        df = df.replace('Station', pd.NaT)
        df = df.replace('ET', pd.NaT)
        df = df.replace('SuMoTuWeTh',pd.NaT )
        df = df.replace('MoTuWeTh', pd.NaT)
        df = df.replace('MoTuWeThFr', pd.NaT)
        df = df.replace('SuMoTuWeThFr', pd.NaT)
        df = df.replace('Sa', pd.NaT)
        df = df.replace('SaSu', pd.NaT)
        df = df.replace('Su', pd.NaT)
    
        # Augment with columns containing actual date of departure and arrival 
        origination_date = pd.to_datetime(date_parsed)
        
        df['Origin Date'] = origination_date
        df['Train Number'] = train_number_extracted
        
        departure_day_offset = pd.to_numeric(df['Schedule Departure Day'], errors='coerce') - 1
        arrival_day_offset = pd.to_numeric(df['Schedule Arrival Day'], errors='coerce') - 1
    
        df['Scheduled Departure Date'] = origination_date + pd.to_timedelta(departure_day_offset, unit='d')
        df['Scheduled Arrival Date'] = origination_date + pd.to_timedelta(arrival_day_offset, unit='d')
    
        # Augment with columns for actual datetime 
        df['Parsed Schedule Departure Time'] = df['Schedule Departure Time'].apply(parse_custom_time)
        df['Parsed Schedule Arrival Time'] = df['Schedule Arrival Time'].apply(parse_custom_time)
        df['Parsed Actual Departure Time'] = df['Actual Departure Time'].apply(parse_custom_time)
        df['Parsed Actual Arrival Time'] = df['Actual Arrival Time'].apply(parse_custom_time)
        
        df['Scheduled Departure Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Departure Date']).split()[0] + ' ' + str(row['Parsed Schedule Departure Time'])) 
            if pd.notna(row['Scheduled Departure Date']) and pd.notna(row['Parsed Schedule Departure Time'])
            else pd.NaT, axis=1)
        
        df['Scheduled Arrival Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Arrival Date']).split()[0] + ' ' + str(row['Parsed Schedule Arrival Time'])) 
            if pd.notna(row['Scheduled Arrival Date']) and pd.notna(row['Parsed Schedule Arrival Time'])
            else pd.NaT, axis=1)
        
        df['Actual Departure Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Departure Date']).split()[0] + ' ' + str(row['Parsed Actual Departure Time'])) 
            if pd.notna(row['Scheduled Departure Date']) and pd.notna(row['Parsed Actual Departure Time'])
            else pd.NaT, axis=1)
        
        df['Actual Arrival Datetime'] = df.apply(
            lambda row: pd.to_datetime(str(row['Scheduled Arrival Date']).split()[0] + ' ' + str(row['Parsed Actual Arrival Time'])) 
            if pd.notna(row['Scheduled Arrival Date']) and pd.notna(row['Parsed Actual Arrival Time'])
            else pd.NaT, axis=1)
    
        # Output cleaned + augmented data
        df = df[['Origin Date', 'Train Number', 'Service Disruption', 'Cancellation', 'Station Code','Scheduled Departure Datetime', 'Scheduled Arrival Datetime', 'Actual Departure Datetime', 'Actual Arrival Datetime', 'Comments']]
        output_filepath = os.path.join(output_dir, filename)                   
        df.to_csv(output_filepath, index=False)
    except ValueError as e:
        exceptions_1.append([filepath, e])
        print(df)
    except IndexError as e:
        exceptions_2.append([filepath, e])
    

In [16]:
len(exceptions_1) # cool beans ok we're done

0